<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#LGBMClassifier" data-toc-modified-id="LGBMClassifier-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LGBMClassifier</a></span></li><li><span><a href="#RidgeClassifier" data-toc-modified-id="RidgeClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>RidgeClassifier</a></span></li></ul></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Тестирование</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#BERT" data-toc-modified-id="BERT-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Проект для «Викишоп» с BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
# загрузка библиотек
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import re
import transformers
import torch
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, GridSearchCV

from lightgbm import LGBMClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier

from tqdm import notebook

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\79853\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\79853\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\79853\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\79853\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\79853\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# чтение файла с данными и сохранение в переменную df
try:
    df = pd.read_csv('C:/Users/79853/Documents/datascience/yandex/Спринт14/toxic_comments.csv')
except:
    df = pd.read_csv('/datasets/toxic_comments.csv') # загрузка онлайн

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
toxic,159571.0,0.101679,0.302226,0.0,0.0,0.0,0.0,1.0


In [5]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
for i in range(11, 15):
    print("\033[1m" + str(i) + "\033[0;0m")
    print(df['text'][i])
    print()

11
bbq 

be a man and lets discuss it-maybe over the phone?

12
Hey... what is it..
@ | talk .
What is it... an exclusive group of some WP TALIBANS...who are good at destroying, self-appointed purist who GANG UP any one who asks them questions abt their ANTI-SOCIAL and DESTRUCTIVE (non)-contribution at WP?

Ask Sityush to clean up his behavior than issue me nonsensical warnings...

13
Before you start throwing accusations and warnings at me, lets review the edit itself-making ad hominem attacks isn't going to strengthen your argument, it will merely make it look like you are abusing your power as an admin. 
Now, the edit itself is relevant-this is probably the single most talked about event int he news as of late. His absence is notable, since he is the only living ex-president who did not attend. That's certainly more notable than his dedicating an aircracft carrier. 
I intend to revert this edit, in hopes of attracting the attention of an admin that is willing to look at the issue it

**Признаки:**
- **text** — текст комментария, 
- **toxic** — целевой признак.

In [7]:
# процентное соотношение значений 0 и 1 в целевом признаке
df['toxic'].value_counts()/df.shape[0]*100

0    89.832112
1    10.167888
Name: toxic, dtype: float64

In [8]:
# подсчёт явных дубликатов
df.duplicated().sum()

0

Текст комментариев на английском языке. Соотношение токсичных комментариев к нетоксичным 1:9. При обучении моделей можно использовать гиперпараметр для борьбы с дисбалансом классов.

В данных нет пропусков и дубликатов.

In [9]:
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [10]:
# очистка текста от цифр, несловесных и одиночных символов и т.д.
def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d", "", text)               # удаление цифр
    text = re.sub(r"\W", " ", text, flags=re.I)  # удаление всех несловесных символов     
    text = re.sub(r"\s+[a-z]\s+", " ", text)     # удаление одиночных символов
    text = re.sub(r"_", " ", text)               # удаление знака подчеркивания
    text = re.sub(r"\s+"," ", text, flags=re.I)  # удаление нескольких пробелов
    text = re.sub(r"^\s", "", text)              # удаление пробелов из начала строки
    text = re.sub(r"\s+$", "", text)             # удаление пробелов с конца строки
        
    return text

In [11]:
# лемматизация текста
def lemma(text):
    tokens = nltk.word_tokenize(text)
    for words in tokens:
        lemmatizer.lemmatize(words)
        
    tagged = nltk.pos_tag(tokens)  # таггетированный текст
    
    # удаление из текста предлогов, союзов, частиц и др.
    tokens_clean = []
    words_del = ['CC', 'DT', 'EX', 'IN', 'RP', 'SYM', 'TO', 'FW']
    """
        CC - Coordinating conjunction
        DT - Determiner
        EX - Existential there
        IN - Preposition or subordinating conjunction
        RP - Particle
        SYM - Symbol
        TO - to
        FW - Foreign word
    """
    for i in range(len(tokens)):
        if tagged[i][1] not in words_del:
            tokens_clean.append(tokens[i])
        
    return tokens_clean

In [12]:
# функция для подготовки датасета, разбиение на train, test и features, target

def prepare_data(data, sample_size, test_size, target, text):
    
    # формирование подвыборки из датасета
    data = data.sample(sample_size).reset_index(drop=True)
    
    data[text] = data[text].transform(lambda x: clean_text(x))  # очистка текста
    data[text] = data[text].transform(lambda x: lemma(x))       # лемматизация текста
    
    # обрезка строк по средней длине строки
    df['text_len'] = df[text].transform(lambda x: len(x))
    mean_len = int(df['text_len'].mean())
    df[text] = df.text.str[:mean_len]
    
    data[text] = data[text].transform(lambda x: ' '.join(x))
    
    # разбиваем датасет на тренировочную и тестовую выборку
    train, test = train_test_split(data, test_size=test_size, random_state=123)
    
    train_features = train[text]
    train_target = train[target]
    test_features = test[text]
    test_target = test[target]
    
    # получение TF-IDF для N-грамм корпуса текста
    count_tf_idf = TfidfVectorizer(stop_words=stopwords, ngram_range=(1, 3), min_df=3, max_df=0.8, sublinear_tf=True)
    
    train_features = count_tf_idf.fit_transform(train_features)
    test_features = count_tf_idf.transform(test_features)
    
    return train_features, train_target, test_features, test_target

## Обучение

Подберем на кроссвалидации гиперпараметры для трех моделей:
- LGBMClassifier(),
- LogisticRegression(),
- RidgeClassifier().

In [13]:
# функция для определения гиперпараметров модели
def grid_search(model, params):
    grid_cv = GridSearchCV(estimator=model,
                           param_grid=params,
                           cv=4,
                           n_jobs=-1,
                           scoring='f1')
    
    start_fit_time = time.time()
    grid_cv.fit(train_features, train_target)
    crossval_time = time.time() - start_fit_time
    
    print('Гиперпараметры: ', grid_cv.best_params_)
    
    global a
    columns = ['model_name', 'model', 'time_sec', 'F1', 'process']
    rows = [['', model, round(crossval_time, 1), round(grid_cv.best_score_, 3), 'кроссвалидация']]
    a = pd.DataFrame(data=rows, columns=columns)
    
    return grid_cv.best_params_

In [14]:
# таблица для накопления результатов
columns = ['model_name', 'model', 'time_sec', 'F1', 'process']
result_tbl = pd.DataFrame(columns=columns)
result_tbl

,model_name,model,time_sec,F1,process


Значения полей таблицы с результатами:
- **model_name** – название модели,
- **model** – конфигурация модели,
- **time_sec** – время обучения/тестирования модели в секундах,
- **F1** – метрика качества F1,
- **process** - обучение/тестирование.

In [15]:
%%time
train_features, train_target, test_features, test_target = prepare_data(df, sample_size=100000,
                                                                        test_size=0.25,
                                                                        target='toxic',
                                                                        text='text')

Wall time: 7min 33s


### LogisticRegression

In [16]:
best1 = grid_search(LogisticRegression(), params={'class_weight': [None, 'balanced'],
                                                  'C': [10, 1.0, 0.1],
                                                  'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                                                  'penalty': ['l1', 'l2', 'elasticnet'],
                                                  'random_state': [123],
                                                  'n_jobs': [-1]
                                                 })
result_tbl = pd.concat([result_tbl, a], axis = 0) # добавление показателей в общую таблицу

C:\Users\79853\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
192 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
24 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\79853\miniconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\79853\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\Users\79853\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError

Гиперпараметры:  {'C': 10, 'class_weight': 'balanced', 'n_jobs': -1, 'penalty': 'l1', 'random_state': 123, 'solver': 'saga'}


C:\Users\79853\miniconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [17]:
logreg = LogisticRegression(**best1)

### LGBMClassifier

In [18]:
best = grid_search(LGBMClassifier(), params={'class_weight': [None, 'balanced'],
                                             'learning_rate': [0.05, 0.1],
                                             'n_estimators': [100, 300],
                                             'random_state': [123],
                                             'n_jobs': [-1]
                                            })
result_tbl = pd.concat([result_tbl, a], axis = 0) # добавление показателей в общую таблицу

Гиперпараметры:  {'class_weight': None, 'learning_rate': 0.1, 'n_estimators': 300, 'n_jobs': -1, 'random_state': 123}


In [19]:
lgbmclf = LGBMClassifier(**best)

### RidgeClassifier

In [20]:
best2 = grid_search(RidgeClassifier(), params={'class_weight': [None, 'balanced'],
                                               'alpha': [0.1, 0.4, 0.7, 1.0],
                                               'fit_intercept': [True, False],
                                               'random_state': [123]
                                               })
result_tbl = pd.concat([result_tbl, a], axis = 0) # добавление показателей в общую таблицу

Гиперпараметры:  {'alpha': 0.4, 'class_weight': None, 'fit_intercept': True, 'random_state': 123}


In [21]:
ridclf = RidgeClassifier(**best2)

In [22]:
result_tbl

,model_name,model,time_sec,F1,process
0,,LogisticRegression(),932.0,0.773,кроссвалидация
0,,LGBMClassifier(),704.2,0.759,кроссвалидация
0,,RidgeClassifier(),63.9,0.704,кроссвалидация


## Тестирование

Протестируем модель логистической регрессии, у которой самое высокое значение метрики F1 на кроссвалидации.

In [23]:
# функция для тестирования модели
def test(model, name):
    
    start_fit_time = time.time()
    model.fit(train_features, train_target)
    fit_time = time.time() - start_fit_time
    
    predictions = model.predict(train_features)
    
    start_predict_time = time.time()
    predictions2 = model.predict(test_features)
    predict_time = time.time() - start_predict_time
        
    global a
    columns = ['model_name', 'model', 'time_sec', 'F1', 'process']
    rows = [[name, model, round(fit_time, 1), round(f1_score(train_target, predictions), 3), 'обучение'],
            [name, model, round(predict_time, 1), round(f1_score(test_target, predictions2), 3), 'тестирование']]
    a = pd.DataFrame(data=rows, columns=columns)
    
    return predictions

In [24]:
predictions = test(logreg, name='logreg')
result_tbl = pd.concat([result_tbl, a], axis = 0) # добавление показателей в общую таблицу

C:\Users\79853\miniconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [25]:
result_tbl

,model_name,model,time_sec,F1,process
0,,LogisticRegression(),932.0,0.773,кроссвалидация
0,,LGBMClassifier(),704.2,0.759,кроссвалидация
0,,RidgeClassifier(),63.9,0.704,кроссвалидация
0,logreg,"LogisticRegression(C=10, class_weight='balance...",362.9,0.969,обучение
1,logreg,"LogisticRegression(C=10, class_weight='balance...",0.0,0.774,тестирование


### Вывод

Логистическая регрессия показала лучшие результаты среди исследованных моделей, в ходе тестирования удалось достичь необходимого значения метрики качества F1 выше 0,75. 

Далее попробуем использовать нейронную сеть Bert.

## BERT

In [26]:
df_bert = df.sample(6000).reset_index(drop=True)

In [27]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel,
                                                    transformers.DistilBertTokenizer,
                                                    'distilbert-base-uncased')

In [28]:
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
tokenized = df_bert['text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

In [30]:
padded = np.array([i + [0]*(512 - len(i)) for i in tokenized.values])

In [31]:
attention_mask = np.where(padded != 0, 1, 0)

In [32]:
%%time
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/60 [00:00<?, ?it/s]

Wall time: 47min 34s


In [33]:
features = np.concatenate(embeddings)
target = df_bert['toxic']

In [34]:
# индекс для отделения тестовой выборки
test_index = int(len(df_bert)*(1-0.25))
test_index

4500

In [35]:
train_features = features[:test_index]
train_target = target[:test_index]

test_features = features[test_index:]
test_target = target[test_index:]

In [36]:
logistic_Bert = LogisticRegression()

In [37]:
predictions3 = test(logistic_Bert, name='logistic_Bert')
result_tbl = pd.concat([result_tbl, a], axis = 0) # добавление показателей в общую таблицу

C:\Users\79853\miniconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [38]:
result_tbl

,model_name,model,time_sec,F1,process
0,,LogisticRegression(),932.0,0.773,кроссвалидация
0,,LGBMClassifier(),704.2,0.759,кроссвалидация
0,,RidgeClassifier(),63.9,0.704,кроссвалидация
0,logreg,"LogisticRegression(C=10, class_weight='balance...",362.9,0.969,обучение
1,logreg,"LogisticRegression(C=10, class_weight='balance...",0.0,0.774,тестирование
0,logistic_Bert,LogisticRegression(),0.7,0.824,обучение
1,logistic_Bert,LogisticRegression(),0.0,0.763,тестирование


Результаты модели Bert по метрикам качества на обучении и тестировании ниже, чем у логистической регрессии, однако, стоит отметить, что качество данной модели можно повысить увеличением обучающей выборки, но для этого требуется больший объем оперативной памяти. 

## Выводы

В совокупности показателей на обучении и тестировании лучшей является модель:
- **logreg = LogisticRegression(C=10, class_weight='balanced', n_jobs=-1, penalty='l1', random_state=123, solver='saga')**

In [39]:
print("\033[1m" + "Результаты выбранной модели" + "\033[0;0m")
result_tbl.loc[result_tbl.model_name == 'logreg']

Результаты выбранной модели


,model_name,model,time_sec,F1,process
0,logreg,"LogisticRegression(C=10, class_weight='balance...",362.9,0.969,обучение
1,logreg,"LogisticRegression(C=10, class_weight='balance...",0.0,0.774,тестирование


## Общий вывод

В ходе проекта были исследованы данные с комментариями клиентов интернет-магазина к описанию товаров с разметкой о токсичности с целью построения модели для классификации комментариев на позитивные и негативные.

На этапе **подготовки данных** было установлено, что в данных присутсвует дисбаланс: соотношение токсичных комментариев к нетоксичным составляет 1:9. Также на этом этапе была разработана функция, которая формирует подвыборку данных, проводит очистку текста от несловесных символов, одиночных символов, лишних пробелов и т.д., лемматтизирует текст, создает матрицу tfidf со словами и их оценками.

В **п. 2** с помощью GridSearchCV на кроссвалидации были подобраны гиперпараметры для трех моделей: **LogisticRegression(), LGBMClassifier(), RidgeClassifier()**. В качестве обучающих данных была использована матрица tfidf для 100 000 комментариев из исходных данных. Наилучшие результаты метрики качества показала модель логистической регрессии со значением **F1=0,77**. В **п. 3** данная модель была протестирована.

В **п. 4** была исследована модель Bert, которая показала результаты ниже, чем логистическая регрессия, однако для ее обучения потребовалось меньше данных - было использовано 6000 комментариев из исходных данных.

В совокупности показателей на обучении и тестировании лучшей была выбрана модель **logreg = LogisticRegression(C=10, class_weight='balanced', n_jobs=-1, penalty='l1', random_state=123, solver='saga')**:
- F1 на обучающей выборке - 0,97
- F1 на тестовой выборке - 0,77